Position Sizing and Risk Management

In [ ]:
import pandas as pd
import numpy as np

class AdvancedBacktester:
    def __init__(self, initial_capital=100000, risk_per_trade=0.01, atr_multiplier=2.0):
        """
        Advanced backtester with volatility-based position sizing and trailing stops.
        
        Args:
            initial_capital (float): Starting capital
            risk_per_trade (float): Risk per trade as fraction of capital (e.g., 0.01 = 1%)
            atr_multiplier (float): ATR multiplier for stop loss (e.g., 2.0 = 2x ATR)
        """
        self.initial_capital = initial_capital
        self.risk_per_trade = risk_per_trade
        self.atr_multiplier = atr_multiplier
        
    def run_backtest(self, data, signals):
        """
        Run the advanced backtest with volatility-based sizing and trailing stops.
        
        Args:
            data (pd.DataFrame): Market data with 'spy_close', 'High', 'Low', 'atr' columns
            signals (pd.Series): Trading signals (1 for long, -1 for short, 0 for flat)
        
        Returns:
            dict: Results including equity curve, trades, and metrics
        """
        # Initialize tracking variables
        equity = [self.initial_capital]
        position_size = 0  # In shares
        current_position = 0  # 1 for long, -1 for short, 0 for flat
        entry_price = 0
        stop_loss_price = 0
        trade_log = []  # Track individual trades
        
        # Create series to store daily returns
        daily_returns = pd.Series(index=data.index, dtype=float)
        daily_returns.iloc[0] = 0  # No return on first day
        
        for i in range(1, len(data)):
            current_date = data.index[i]
            current_price = data['spy_close'].iloc[i]
            current_atr = data['atr'].iloc[i]
            
            # Check for exit conditions
            exit_signal = False
            exit_reason = ""
            
            if current_position != 0:
                # Check if stop loss is hit
                if current_position == 1:  # Long position
                    # Check if low price hits stop loss
                    if data['Low'].iloc[i] <= stop_loss_price:
                        exit_signal = True
                        exit_reason = "Stop Loss Hit"
                    # Check if signal turns flat
                    elif signals.iloc[i] == 0:
                        exit_signal = True
                        exit_reason = "Signal Exit"
                elif current_position == -1:  # Short position
                    # Check if high price hits stop loss
                    if data['High'].iloc[i] >= stop_loss_price:
                        exit_signal = True
                        exit_reason = "Stop Loss Hit"
                    # Check if signal turns flat
                    elif signals.iloc[i] == 0:
                        exit_signal = True
                        exit_reason = "Signal Exit"
            
            # Execute exit if needed
            if exit_signal:
                # Calculate P&L
                if current_position == 1:  # Long
                    pnl = (current_price - entry_price) * position_size
                elif current_position == -1:  # Short
                    pnl = (entry_price - current_price) * position_size
                else:
                    pnl = 0
                
                # Update equity
                new_equity = equity[-1] + pnl
                
                # Log the trade
                trade_log.append({
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'entry_price': entry_price,
                    'exit_price': current_price,
                    'position_size': position_size,
                    'pnl': pnl,
                    'return_pct': pnl / (entry_price * position_size) if (entry_price * position_size) != 0 else 0,
                    'exit_reason': exit_reason
                })
                
                # Reset position
                current_position = 0
                position_size = 0
                entry_price = 0
                stop_loss_price = 0
                equity.append(new_equity)
            else:
                # Check for entry conditions
                should_enter_long = (signals.iloc[i] == 1 and current_position == 0)
                should_enter_short = (signals.iloc[i] == -1 and current_position == 0)
                
                if should_enter_long or should_enter_short:
                    # Calculate position size based on volatility
                    target_risk = equity[-1] * self.risk_per_trade  # Target risk amount
                    position_size = int(target_risk / current_atr)  # Position size in shares
                    
                    # Ensure we have enough capital
                    entry_price = current_price
                    required_capital = entry_price * position_size
                    if required_capital > equity[-1]:
                        position_size = int(equity[-1] / entry_price)
                    
                    # Set initial stop loss
                    if should_enter_long:
                        current_position = 1
                        stop_loss_price = entry_price - (self.atr_multiplier * current_atr)
                        entry_date = current_date
                    elif should_enter_short:
                        current_position = -1
                        stop_loss_price = entry_price + (self.atr_multiplier * current_atr)
                        entry_date = current_date
                
                # Calculate daily P&L if in position
                if current_position != 0:
                    if current_position == 1:  # Long position
                        # Update trailing stop
                        new_potential_stop = current_price - (self.atr_multiplier * current_atr)
                        stop_loss_price = max(stop_loss_price, new_potential_stop)
                        
                        # Calculate daily P&L
                        daily_pnl = (current_price - data['spy_close'].iloc[i-1]) * position_size
                    else:  # Short position
                        # Update trailing stop
                        new_potential_stop = current_price + (self.atr_multiplier * current_atr)
                        stop_loss_price = min(stop_loss_price, new_potential_stop)
                        
                        # Calculate daily P&L
                        daily_pnl = (data['spy_close'].iloc[i-1] - current_price) * position_size
                    
                    new_equity = equity[-1] + daily_pnl
                else:
                    # No position, no P&L change
                    new_equity = equity[-1]
                
                equity.append(new_equity)
            
            # Calculate daily return
            if len(equity) > 1:
                daily_returns.iloc[i] = (equity[-1] - equity[-2]) / equity[-2]
        
        # Convert equity list to series with proper index
        equity_series = pd.Series(equity, index=data.index[:len(equity)])
        
        return {
            'equity_curve': equity_series,
            'daily_returns': daily_returns,
            'trade_log': pd.DataFrame(trade_log),
            'final_capital': equity[-1] if equity else self.initial_capital
        }

def calculate_performance_metrics(returns):
    """Calculate key performance metrics."""
    total_return = (1 + returns).prod() - 1
    annual_return = returns.mean() * 252
    annual_volatility = returns.std() * np.sqrt(252)
    
    if annual_volatility != 0:
        sharpe_ratio = annual_return / annual_volatility
    else:
        sharpe_ratio = np.nan
    
    # Calculate drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'annual_volatility': annual_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

# Example usage:
if __name__ == '__main__':
    # Import all necessary functions from previous scripts
    import yfinance as yf
    import matplotlib.pyplot as plt

    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    def generate_vol_norm_rsi_signals(df, rsi_entry=30, rsi_exit=50, atr_multiplier=1.0, vix_threshold=40):
        """
        Generates trading signals based on the Volatility-Normalised RSI strategy.
        Returns a DataFrame with a 'signal' column (1 for long, -1 for short, 0 for flat).
        """
        signals = pd.DataFrame(index=df.index)
        signals['signal'] = 0

        # Entry Conditions
        long_entry_condition = (df['rsi'] < rsi_entry) & \
                               (df['spy_close'] < (df['spy_close'].shift(1) - (atr_multiplier * df['atr'])))
        
        short_entry_condition = (df['rsi'] > (100 - rsi_entry)) & \
                                (df['spy_close'] > (df['spy_close'].shift(1) + (atr_multiplier * df['atr'])))

        # Exit Conditions
        long_exit_condition = df['rsi'] > rsi_exit
        short_exit_condition = df['rsi'] < (100 - rsi_exit)

        # State machine to hold positions
        position = 0
        for i in range(len(df)):
            if position == 0: # If flat
                if long_entry_condition.iloc[i]:
                    position = 1
                elif short_entry_condition.iloc[i]:
                    position = -1
            elif position == 1: # If long
                if long_exit_condition.iloc[i]:
                    position = 0
            elif position == -1: # If short
                if short_exit_condition.iloc[i]:
                    position = 0
            signals['signal'].iloc[i] = position

        # VIX Regime Filter - Fixed the assignment
        vix_high_mask = df['vix'] > vix_threshold
        signals.loc[vix_high_mask, 'signal'] = 0
        
        return signals['signal']

    # Run the advanced backtest
    market_data = get_market_data()
    signals = generate_vol_norm_rsi_signals(market_data)
    
    # Create and run advanced backtester
    backtester = AdvancedBacktester(initial_capital=100000, risk_per_trade=0.01, atr_multiplier=2.0)
    results = backtester.run_backtest(market_data, signals)
    
    # Display results
    print("Advanced Backtest Results:")
    print(f"Final Capital: ${results['final_capital']:,.2f}")
    print(f"Total Return: {(results['final_capital'] - backtester.initial_capital) / backtester.initial_capital:.2%}")
    
    # Calculate performance metrics
    metrics = calculate_performance_metrics(results['daily_returns'].dropna())
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
    
    # Show trade statistics
    trade_log = results['trade_log']
    if not trade_log.empty:
        print(f"\nNumber of Trades: {len(trade_log)}")
        print(f"Win Rate: {((trade_log['pnl'] > 0).sum() / len(trade_log)):.2%}")
        print(f"Average P&L per Trade: ${trade_log['pnl'].mean():.2f}")
    
    # Plot equity curve
    plt.figure(figsize=(15, 7))
    plt.plot(results['equity_curve'].index, results['equity_curve'], label='Equity Curve')
    plt.title('Advanced Backtest - Equity Curve with Volatility-Based Position Sizing')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True)
    plt.show()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\USER\AppData\Local\Temp\ipykernel_36444\207009521.py:295: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Advanced Backtest Results:
Final Capital: $33,759.81
Total Return: -66.24%
Sharpe Ratio: -0.62
Max Drawdown: -67.58%

Number of Trades: 228
Win Rate: 43.42%
Average P&L per Trade: $-38.04


In [ ]:
import pandas as pd
import numpy as np

class AdvancedBacktester:
    def __init__(self, initial_capital=100000, risk_per_trade=0.01, atr_multiplier=2.0):
        """
        Advanced backtester with volatility-based position sizing and trailing stops.
        
        Args:
            initial_capital (float): Starting capital
            risk_per_trade (float): Risk per trade as fraction of capital (e.g., 0.01 = 1%)
            atr_multiplier (float): ATR multiplier for stop loss (e.g., 2.0 = 2x ATR)
        """
        self.initial_capital = initial_capital
        self.risk_per_trade = risk_per_trade
        self.atr_multiplier = atr_multiplier
        
    def run_backtest(self, data, signals):
        """
        Run the advanced backtest with volatility-based sizing and trailing stops.
        
        Args:
            data (pd.DataFrame): Market data with 'spy_close', 'High', 'Low', 'atr' columns
            signals (pd.Series): Trading signals (1 for long, -1 for short, 0 for flat)
        
        Returns:
            dict: Results including equity curve, trades, and metrics
        """
        # Initialize tracking variables
        equity = [self.initial_capital]
        position_size = 0  # In shares
        current_position = 0  # 1 for long, -1 for short, 0 for flat
        entry_price = 0
        stop_loss_price = 0
        trade_log = []  # Track individual trades
        
        # Create series to store daily returns
        daily_returns = pd.Series(index=data.index, dtype=float)
        daily_returns.iloc[0] = 0  # No return on first day
        
        for i in range(1, len(data)):
            current_date = data.index[i]
            current_price = data['spy_close'].iloc[i]
            current_atr = data['atr'].iloc[i]
            
            # Check for exit conditions
            exit_signal = False
            exit_reason = ""
            
            if current_position != 0:
                # Check if stop loss is hit
                if current_position == 1:  # Long position
                    # Check if low price hits stop loss
                    if data['Low'].iloc[i] <= stop_loss_price:
                        exit_signal = True
                        exit_reason = "Stop Loss Hit"
                    # Check if signal turns flat
                    elif signals.iloc[i] == 0:
                        exit_signal = True
                        exit_reason = "Signal Exit"
                elif current_position == -1:  # Short position
                    # Check if high price hits stop loss
                    if data['High'].iloc[i] >= stop_loss_price:
                        exit_signal = True
                        exit_reason = "Stop Loss Hit"
                    # Check if signal turns flat
                    elif signals.iloc[i] == 0:
                        exit_signal = True
                        exit_reason = "Signal Exit"
            
            # Execute exit if needed
            if exit_signal:
                # Calculate P&L
                if current_position == 1:  # Long
                    pnl = (current_price - entry_price) * position_size
                elif current_position == -1:  # Short
                    pnl = (entry_price - current_price) * position_size
                else:
                    pnl = 0
                
                # Update equity
                new_equity = equity[-1] + pnl
                
                # Log the trade
                trade_log.append({
                    'entry_date': entry_date,
                    'exit_date': current_date,
                    'entry_price': entry_price,
                    'exit_price': current_price,
                    'position_size': position_size,
                    'pnl': pnl,
                    'return_pct': pnl / (entry_price * position_size) if (entry_price * position_size) != 0 else 0,
                    'exit_reason': exit_reason
                })
                
                # Reset position
                current_position = 0
                position_size = 0
                entry_price = 0
                stop_loss_price = 0
                equity.append(new_equity)
            else:
                # Check for entry conditions
                should_enter_long = (signals.iloc[i] == 1 and current_position == 0)
                should_enter_short = (signals.iloc[i] == -1 and current_position == 0)
                
                if should_enter_long or should_enter_short:
                    # Calculate position size based on volatility
                    target_risk = equity[-1] * self.risk_per_trade  # Target risk amount
                    position_size = int(target_risk / current_atr)  # Position size in shares
                    
                    # Ensure we have enough capital
                    entry_price = current_price
                    required_capital = entry_price * position_size
                    if required_capital > equity[-1]:
                        position_size = int(equity[-1] / entry_price)
                    
                    # Set initial stop loss
                    if should_enter_long:
                        current_position = 1
                        stop_loss_price = entry_price - (self.atr_multiplier * current_atr)
                        entry_date = current_date
                    elif should_enter_short:
                        current_position = -1
                        stop_loss_price = entry_price + (self.atr_multiplier * current_atr)
                        entry_date = current_date
                
                # Calculate daily P&L if in position
                if current_position != 0:
                    if current_position == 1:  # Long position
                        # Update trailing stop
                        new_potential_stop = current_price - (self.atr_multiplier * current_atr)
                        stop_loss_price = max(stop_loss_price, new_potential_stop)
                        
                        # Calculate daily P&L
                        daily_pnl = (current_price - data['spy_close'].iloc[i-1]) * position_size
                    else:  # Short position
                        # Update trailing stop
                        new_potential_stop = current_price + (self.atr_multiplier * current_atr)
                        stop_loss_price = min(stop_loss_price, new_potential_stop)
                        
                        # Calculate daily P&L
                        daily_pnl = (data['spy_close'].iloc[i-1] - current_price) * position_size
                    
                    new_equity = equity[-1] + daily_pnl
                else:
                    # No position, no P&L change
                    new_equity = equity[-1]
                
                equity.append(new_equity)
            
            # Calculate daily return
            if len(equity) > 1:
                daily_returns.iloc[i] = (equity[-1] - equity[-2]) / equity[-2]
        
        # Convert equity list to series with proper index
        equity_series = pd.Series(equity, index=data.index[:len(equity)])
        
        return {
            'equity_curve': equity_series,
            'daily_returns': daily_returns,
            'trade_log': pd.DataFrame(trade_log),
            'final_capital': equity[-1] if equity else self.initial_capital
        }

def calculate_performance_metrics(returns):
    """Calculate key performance metrics."""
    total_return = (1 + returns).prod() - 1
    annual_return = returns.mean() * 252
    annual_volatility = returns.std() * np.sqrt(252)
    
    if annual_volatility != 0:
        sharpe_ratio = annual_return / annual_volatility
    else:
        sharpe_ratio = np.nan
    
    # Calculate drawdown
    cumulative = (1 + returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'total_return': total_return,
        'annual_return': annual_return,
        'annual_volatility': annual_volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown
    }

# Example usage:
if __name__ == '__main__':
    # Import all necessary functions from previous scripts
    import yfinance as yf
    import matplotlib.pyplot as plt

    def calculate_features(df):
        """Engineers all necessary features from the raw OHLCV data."""
        df = df.copy()
        
        # 1. Relative Strength Index (RSI) - Fixed to handle division by zero
        delta = df['spy_close'].diff()  # Use spy_close instead of 'Close'
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        
        # Handle division by zero and NaN values
        rs = np.where((loss == 0) | (loss.isna()), np.nan, gain / loss)
        df['rsi'] = 100 - (100 / (1 + rs))
        
        # 2. Bollinger Bands
        df['sma_20'] = df['spy_close'].rolling(window=20).mean()
        df['std_20'] = df['spy_close'].rolling(window=20).std()
        df['bollinger_upper'] = df['sma_20'] + (df['std_20'] * 2)
        df['bollinger_lower'] = df['sma_20'] - (df['std_20'] * 2)

        # 3. Rolling Z-Score
        df['z_score_20'] = (df['spy_close'] - df['sma_20']) / df['std_20']

        # 4. Average True Range (ATR) - Fixed column references
        high_low = df['High'] - df['Low']
        high_close = np.abs(df['High'] - df['spy_close'].shift())  # Use spy_close
        low_close = np.abs(df['Low'] - df['spy_close'].shift())    # Use spy_close
        tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
        df['atr'] = tr.rolling(window=14).mean()

        # 5. Add Volume Moving Average
        df['volume_sma_50'] = df['Volume'].rolling(window=50).mean()
        
        return df

    def get_market_data(start_date="2005-01-01"):
        """Downloads SPY and VIX data and engineers features."""
        spy_data = yf.download('SPY', start=start_date, auto_adjust=True)
        vix_data = yf.download('^VIX', start=start_date, auto_adjust=True)

        # Reset any potential multi-level indexes and ensure clean column structure
        if isinstance(spy_data.columns, pd.MultiIndex):
            spy_data.columns = spy_data.columns.droplevel(1)  # Remove the second level if it exists
        if isinstance(vix_data.columns, pd.MultiIndex):
            vix_data.columns = vix_data.columns.droplevel(1)  # Remove the second level if it exists
        
        # Select only the columns we need
        spy_data = spy_data[['Open', 'High', 'Low', 'Close', 'Volume']].copy()
        vix_data = vix_data[['Close']].copy()
        vix_data = vix_data.rename(columns={'Close': 'vix'})
        
        # Align the date indices properly
        common_dates = spy_data.index.intersection(vix_data.index)
        spy_data = spy_data.loc[common_dates]
        vix_data = vix_data.loc[common_dates]
        
        # Combine into a single DataFrame using merge
        df = spy_data.copy()
        df['vix'] = vix_data['vix']
        df.rename(columns={'Close': 'spy_close'}, inplace=True)  # Rename for clarity

        # Calculate features using the function
        df = calculate_features(df)
        
        # Calculate daily returns for the backtest
        df['daily_return'] = df['spy_close'].pct_change()

        return df.dropna()

    def generate_sma_exit_signals(df, rsi_entry=25, atr_multiplier=1.5, vix_threshold=40):

        signals = pd.DataFrame(index=df.index)
        signals['signal'] = 0.0
        position = 0  # 0: flat, 1: long, -1: short
    
        # --- ENTRY CONDITIONS (Modified with Volume) ---
        long_entry = (df['rsi'] < rsi_entry) & \
                     (df['spy_close'] < df['bollinger_lower']) & \
                     (df['Volume'] > df['volume_sma_50'] * 1.5) 
        
        short_entry = (df['rsi'] > (100 - rsi_entry)) & \
                      (df['spy_close'] > df['bollinger_upper']) & \
                      (df['Volume'] > df['volume_sma_50'] * 1.5)
    
        for i in range(1, len(df)):
            # --- EXIT CONDITIONS ---
            # Exit long if price crosses above the 20-day SMA
            if position == 1 and (df['spy_close'].iloc[i] > df['sma_20'].iloc[i]):
                position = 0
            # Exit short if price crosses below the 20-day SMA
            elif position == -1 and (df['spy_close'].iloc[i] < df['sma_20'].iloc[i]):
                position = 0
            
            # --- ENTRY LOGIC ---
            if position == 0:
                if long_entry.iloc[i]:
                    position = 1
                elif short_entry.iloc[i]:
                    position = 0 # stop shorting
            
            signals['signal'].iloc[i] = position
    
        # VIX Regime Filter
        signals.loc[df['vix'] > vix_threshold, 'signal'] = 0
        
        return signals['signal']

    # Run the advanced backtest
    market_data = get_market_data()
    signals = generate_sma_exit_signals(market_data)
    
    # Create and run advanced backtester
    backtester = AdvancedBacktester(initial_capital=100000, risk_per_trade=0.01, atr_multiplier=2.0)
    results = backtester.run_backtest(market_data, signals)
    
    # Display results
    print("Advanced Backtest Results:")
    print(f"Final Capital: ${results['final_capital']:,.2f}")
    print(f"Total Return: {(results['final_capital'] - backtester.initial_capital) / backtester.initial_capital:.2%}")
    
    # Calculate performance metrics
    metrics = calculate_performance_metrics(results['daily_returns'].dropna())
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"Max Drawdown: {metrics['max_drawdown']:.2%}")
    
    # Show trade statistics
    trade_log = results['trade_log']
    if not trade_log.empty:
        print(f"\nNumber of Trades: {len(trade_log)}")
        print(f"Win Rate: {((trade_log['pnl'] > 0).sum() / len(trade_log)):.2%}")
        print(f"Average P&L per Trade: ${trade_log['pnl'].mean():.2f}")
    
    # Plot equity curve
    plt.figure(figsize=(15, 7))
    plt.plot(results['equity_curve'].index, results['equity_curve'], label='Equity Curve')
    plt.title('Advanced Backtest - Equity Curve with Volatility-Based Position Sizing')
    plt.xlabel('Date')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True)
    plt.show()